In [66]:
!pip install requests


In [67]:
import json
import requests

response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)

for public_holiday in public_holidays:
  print(public_holiday['date'])


2024-01-01
2024-02-12
2024-02-13
2024-03-29
2024-03-31
2024-04-21
2024-05-01
2024-05-30
2024-07-09
2024-09-07
2024-10-12
2024-11-02
2024-11-15
2024-11-20
2024-12-25


In [68]:
feriados_br = public_holidays
feriados_br = [feriado['date'] for feriado in feriados_br]

import pandas as pd

feriados_br = pd.DataFrame(feriados_br, columns=['date'])
feriados_br


,date
0,2024-01-01
1,2024-02-12
2,2024-02-13
3,2024-03-29
4,2024-03-31
5,2024-04-21
6,2024-05-01
7,2024-05-30
8,2024-07-09
9,2024-09-07


In [83]:
#1 quantos feriados há no brasil em todo o ano de 2024?

feriados_br.count()

,0
date,15
month,15
weekday,15


In [70]:
#2 qual mês de 2024 tem o maior número de feriados?

feriados_br['month'] = pd.to_datetime(feriados_br['date']).dt.month
feriados_br['month'].value_counts()

,count
month,
11,3
2,2
3,2
5,2
1,1
4,1
7,1
9,1
10,1


In [71]:
#3 quanto feriados em 2024 caem em dias da semana (segunda a sexta)?

feriados_br['weekday'] = pd.to_datetime(feriados_br['date']).dt.weekday
feriados_br['weekday'].value_counts()

,count
weekday,
2,3
5,3
0,2
1,2
4,2
6,2
3,1


In [72]:
!pip install openmeteo-requests

In [73]:
!pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.5 MB/s eta 0:00:00


In [74]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822,
	"start_date": "2024-01-01",
	"end_date": "2024-08-01",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min"],
	"timezone": "America/Sao_Paulo"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe


Coordinates -22.75°N -43.0°E
Elevation 12.0 m asl
Timezone b'America/Sao_Paulo' b'-03'
Timezone difference to GMT+0 -10800 s


,date,weather_code,temperature_2m_max,temperature_2m_min
0,2024-01-01 03:00:00+00:00,61.0,27.625999,22.925999
1,2024-01-02 03:00:00+00:00,80.0,29.175999,22.976000
2,2024-01-03 03:00:00+00:00,80.0,30.175999,23.476000
3,2024-01-04 03:00:00+00:00,95.0,28.125999,23.825998
4,2024-01-05 03:00:00+00:00,80.0,28.675999,24.476000
...,...,...,...,...
209,2024-07-28 03:00:00+00:00,3.0,33.625999,17.875999
210,2024-07-29 03:00:00+00:00,45.0,27.625999,17.726000
211,2024-07-30 03:00:00+00:00,95.0,23.075998,19.025999
212,2024-07-31 03:00:00+00:00,45.0,24.375999,18.025999


In [75]:
#4 qual foi a temperatura média em cada mês?

#criar coluna para temperatura média diária:
daily_dataframe['temperature_mean'] = (daily_dataframe['temperature_2m_max'] + daily_dataframe['temperature_2m_min']) / 2

#agrupar por mês
daily_dataframe['month'] = daily_dataframe['date'].dt.month
daily_dataframe.groupby('month')['temperature_mean'].mean()

,temperature_mean
month,
1,27.962290
2,28.510483
3,27.446966
4,26.680166
5,26.109871
6,23.961832
7,22.242128
8,22.000999


In [38]:
#abrindo arquivo json com as weather codes

json.load(open('/content/descriptions.json'))

{'0': {'day': {'description': 'Sunny',
   'image': 'http://openweathermap.org/img/wn/01d@2x.png'},
  'night': {'description': 'Clear',
   'image': 'http://openweathermap.org/img/wn/01n@2x.png'}},
 '1': {'day': {'description': 'Mainly Sunny',
   'image': 'http://openweathermap.org/img/wn/01d@2x.png'},
  'night': {'description': 'Mainly Clear',
   'image': 'http://openweathermap.org/img/wn/01n@2x.png'}},
 '2': {'day': {'description': 'Partly Cloudy',
   'image': 'http://openweathermap.org/img/wn/02d@2x.png'},
  'night': {'description': 'Partly Cloudy',
   'image': 'http://openweathermap.org/img/wn/02n@2x.png'}},
 '3': {'day': {'description': 'Cloudy',
   'image': 'http://openweathermap.org/img/wn/03d@2x.png'},
  'night': {'description': 'Cloudy',
   'image': 'http://openweathermap.org/img/wn/03n@2x.png'}},
 '45': {'day': {'description': 'Foggy',
   'image': 'http://openweathermap.org/img/wn/50d@2x.png'},
  'night': {'description': 'Foggy',
   'image': 'http://openweathermap.org/img/wn/50

In [60]:
#importando para o pandas

import pandas as pd
import json
import pandas.io.json
from pandas import json_normalize


teste = pd.read_json('/content/descriptions.json', orient='index')
teste



,day,night
0,"{'description': 'Sunny', 'image': 'http://open...","{'description': 'Clear', 'image': 'http://open..."
1,"{'description': 'Mainly Sunny', 'image': 'http...","{'description': 'Mainly Clear', 'image': 'http..."
2,"{'description': 'Partly Cloudy', 'image': 'htt...","{'description': 'Partly Cloudy', 'image': 'htt..."
3,"{'description': 'Cloudy', 'image': 'http://ope...","{'description': 'Cloudy', 'image': 'http://ope..."
45,"{'description': 'Foggy', 'image': 'http://open...","{'description': 'Foggy', 'image': 'http://open..."
48,"{'description': 'Rime Fog', 'image': 'http://o...","{'description': 'Rime Fog', 'image': 'http://o..."
51,"{'description': 'Light Drizzle', 'image': 'htt...","{'description': 'Light Drizzle', 'image': 'htt..."
53,"{'description': 'Drizzle', 'image': 'http://op...","{'description': 'Drizzle', 'image': 'http://op..."
55,"{'description': 'Heavy Drizzle', 'image': 'htt...","{'description': 'Heavy Drizzle', 'image': 'htt..."
56,"{'description': 'Light Freezing Drizzle', 'ima...","{'description': 'Light Freezing Drizzle', 'ima..."


In [43]:
#extraindo as informações da coluna day

day_df = json_normalize(teste['day'],
                       sep=',',
                       meta= ['description', 'image'],
                       meta_prefix='d_')

day_df


,description,image
0,Sunny,http://openweathermap.org/img/wn/01d@2x.png
1,Mainly Sunny,http://openweathermap.org/img/wn/01d@2x.png
2,Partly Cloudy,http://openweathermap.org/img/wn/02d@2x.png
3,Cloudy,http://openweathermap.org/img/wn/03d@2x.png
4,Foggy,http://openweathermap.org/img/wn/50d@2x.png
5,Rime Fog,http://openweathermap.org/img/wn/50d@2x.png
6,Light Drizzle,http://openweathermap.org/img/wn/09d@2x.png
7,Drizzle,http://openweathermap.org/img/wn/09d@2x.png
8,Heavy Drizzle,http://openweathermap.org/img/wn/09d@2x.png
9,Light Freezing Drizzle,http://openweathermap.org/img/wn/09d@2x.png


In [61]:
#renomeando o index em coluna weather_codes em teste

teste = teste.rename_axis('weather_codes').reset_index()
teste



,weather_codes,day,night
0,0,"{'description': 'Sunny', 'image': 'http://open...","{'description': 'Clear', 'image': 'http://open..."
1,1,"{'description': 'Mainly Sunny', 'image': 'http...","{'description': 'Mainly Clear', 'image': 'http..."
2,2,"{'description': 'Partly Cloudy', 'image': 'htt...","{'description': 'Partly Cloudy', 'image': 'htt..."
3,3,"{'description': 'Cloudy', 'image': 'http://ope...","{'description': 'Cloudy', 'image': 'http://ope..."
4,45,"{'description': 'Foggy', 'image': 'http://open...","{'description': 'Foggy', 'image': 'http://open..."
5,48,"{'description': 'Rime Fog', 'image': 'http://o...","{'description': 'Rime Fog', 'image': 'http://o..."
6,51,"{'description': 'Light Drizzle', 'image': 'htt...","{'description': 'Light Drizzle', 'image': 'htt..."
7,53,"{'description': 'Drizzle', 'image': 'http://op...","{'description': 'Drizzle', 'image': 'http://op..."
8,55,"{'description': 'Heavy Drizzle', 'image': 'htt...","{'description': 'Heavy Drizzle', 'image': 'htt..."
9,56,"{'description': 'Light Freezing Drizzle', 'ima...","{'description': 'Light Freezing Drizzle', 'ima..."


In [62]:
#drop colunas day e night

teste = teste.drop(columns=['day', 'night'])
teste

,weather_codes
0,0
1,1
2,2
3,3
4,45
5,48
6,51
7,53
8,55
9,56


In [63]:
#unindo teste e day_df pelo índice

teste = teste.join(day_df)
teste

,weather_codes,description,image
0,0,Sunny,http://openweathermap.org/img/wn/01d@2x.png
1,1,Mainly Sunny,http://openweathermap.org/img/wn/01d@2x.png
2,2,Partly Cloudy,http://openweathermap.org/img/wn/02d@2x.png
3,3,Cloudy,http://openweathermap.org/img/wn/03d@2x.png
4,45,Foggy,http://openweathermap.org/img/wn/50d@2x.png
5,48,Rime Fog,http://openweathermap.org/img/wn/50d@2x.png
6,51,Light Drizzle,http://openweathermap.org/img/wn/09d@2x.png
7,53,Drizzle,http://openweathermap.org/img/wn/09d@2x.png
8,55,Heavy Drizzle,http://openweathermap.org/img/wn/09d@2x.png
9,56,Light Freezing Drizzle,http://openweathermap.org/img/wn/09d@2x.png


In [64]:
#tranformando a coluna weather_codes em float

teste['weather_codes'] = teste['weather_codes'].astype(float)

In [79]:
#unindo daily_dataframe com teste em weather_codes = weather_code

daily_dataframe = daily_dataframe.merge(teste, left_on='weather_code', right_on='weather_codes')
daily_dataframe


,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_mean,month,weather_codes,description,image
0,2024-01-01 03:00:00+00:00,61.0,27.625999,22.925999,25.275999,1,61.0,Light Rain,http://openweathermap.org/img/wn/10d@2x.png
1,2024-02-14 03:00:00+00:00,61.0,32.676003,25.175999,28.926001,2,61.0,Light Rain,http://openweathermap.org/img/wn/10d@2x.png
2,2024-03-27 03:00:00+00:00,61.0,25.625999,22.775999,24.201000,3,61.0,Light Rain,http://openweathermap.org/img/wn/10d@2x.png
3,2024-05-27 03:00:00+00:00,61.0,24.825998,20.275999,22.550999,5,61.0,Light Rain,http://openweathermap.org/img/wn/10d@2x.png
4,2024-01-02 03:00:00+00:00,80.0,29.175999,22.976000,26.076000,1,80.0,Light Showers,http://openweathermap.org/img/wn/09d@2x.png
...,...,...,...,...,...,...,...,...,...
209,2024-06-17 03:00:00+00:00,1.0,32.426003,18.325998,25.375999,6,1.0,Mainly Sunny,http://openweathermap.org/img/wn/01d@2x.png
210,2024-06-19 03:00:00+00:00,1.0,31.476000,17.825998,24.650999,6,1.0,Mainly Sunny,http://openweathermap.org/img/wn/01d@2x.png
211,2024-06-24 03:00:00+00:00,1.0,33.426003,21.275999,27.351002,6,1.0,Mainly Sunny,http://openweathermap.org/img/wn/01d@2x.png
212,2024-06-15 03:00:00+00:00,0.0,32.176003,17.775999,24.976002,6,0.0,Sunny,http://openweathermap.org/img/wn/01d@2x.png


In [82]:
#5 Qual foi o weather code predominante em cada mês?

daily_dataframe.groupby('month')['description'].value_counts()

month  description                  
1      Light Showers                    13
       Thunderstorm                      8
       Cloudy                            3
       Foggy                             3
       Light Thunderstorms With Hail     3
       Light Rain                        1
2      Light Showers                    10
       Thunderstorm                      6
       Cloudy                            5
       Foggy                             4
       Light Thunderstorms With Hail     2
       Partly Cloudy                     1
       Light Rain                        1
3      Thunderstorm                     14
       Foggy                             5
       Light Showers                     4
       Partly Cloudy                     3
       Light Thunderstorms With Hail     2
       Cloudy                            2
       Light Rain                        1
4      Foggy                            16
       Cloudy                            8
       Light Showers                     3
       Partly Cloudy                     2
       Rain                              1
5      Partly Cloudy                     9
       Cloudy                            8
       Foggy                             7
       Light Showers                     5
       Thunderstorm                      1
       Light Rain                        1
6      Foggy                            14
       Light Showers                     6
       Mainly Sunny                      4
       Sunny                             2
       Cloudy                            2
       Partly Cloudy                     2
7      Foggy                            18
       Cloudy                            9
       Light Showers                     2
       Thunderstorm                      1
       Partly Cloudy                     1
8      Foggy                             1
Name: count, dtype: int64

In [87]:
feriados_br['date'] = pd.to_datetime(feriados_br['date'])

In [99]:
#junção da daily_dataframe com feriados_br

#transformar daily_dataframe['date'] em datetime
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])

daily_dataframe1 = daily_dataframe.merge(feriados_br, left_on='date', right_on='date')
daily_dataframe1

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_mean,month_x,weather_codes,description,image,month_y,weekday
0,2024-01-01,61.0,27.625999,22.925999,25.275999,1,61.0,Light Rain,http://openweathermap.org/img/wn/10d@2x.png,1,0
1,2024-02-12,80.0,37.726002,24.925999,31.326000,2,80.0,Light Showers,http://openweathermap.org/img/wn/09d@2x.png,2,0
2,2024-03-29,95.0,29.575998,23.275999,26.425999,3,95.0,Thunderstorm,http://openweathermap.org/img/wn/11d@2x.png,3,4
3,2024-02-13,3.0,37.926003,25.275999,31.601002,2,3.0,Cloudy,http://openweathermap.org/img/wn/03d@2x.png,2,1
4,2024-05-30,3.0,24.425999,18.675999,21.550999,5,3.0,Cloudy,http://openweathermap.org/img/wn/03d@2x.png,5,3
5,2024-07-09,3.0,23.525999,20.226000,21.875999,7,3.0,Cloudy,http://openweathermap.org/img/wn/03d@2x.png,7,1
6,2024-04-21,45.0,29.726000,19.175999,24.451000,4,45.0,Foggy,http://openweathermap.org/img/wn/50d@2x.png,4,6
7,2024-03-31,2.0,29.175999,23.375999,26.275999,3,2.0,Partly Cloudy,http://openweathermap.org/img/wn/02d@2x.png,3,6
8,2024-05-01,2.0,35.976002,22.625999,29.301001,5,2.0,Partly Cloudy,http://openweathermap.org/img/wn/02d@2x.png,5,2


In [100]:
#6 qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

tempo_feriados = daily_dataframe1[['date', 'temperature_mean', 'description']]
tempo_feriados

,date,temperature_mean,description
0,2024-01-01,25.275999,Light Rain
1,2024-02-12,31.326000,Light Showers
2,2024-03-29,26.425999,Thunderstorm
3,2024-02-13,31.601002,Cloudy
4,2024-05-30,21.550999,Cloudy
5,2024-07-09,21.875999,Cloudy
6,2024-04-21,24.451000,Foggy
7,2024-03-31,26.275999,Partly Cloudy
8,2024-05-01,29.301001,Partly Cloudy


In [ ]:
#7 Resposta: apenas os feriados de Páscoa (31/3) e Dia do Trabalhador (01/05) foram aproveitáveis, pois foram parcialmente nublados, possibilitando praia.
#os demais, apesar de apresentarem temperaturas acima de 20 graus, foram de tempo nublado e/ou chuvas e tempestades.

In [ ]:
#8 Resposta: O feriado do dia do Trabalhador foi o mais aproveitável, pois com temperatura amena (29ºC) e com sol.